### Pull Data from USGS

https://earthquake.usgs.gov/fdsnws/event/1/

In [ ]:
import pandas as pd
import folium
from folium import plugins
from io import StringIO
import requests

pd.options.display.max_columns = None


payload = {
    'format': 'csv', 
#     'starttime': None,  # default last 30 days
#     'endtime': '2019-06-03',  # default now
    'minmagnitude': 2.5,  # default null
    'limit': None,  # default null, returns 404 over 20,000
}
url = 'https://earthquake.usgs.gov/fdsnws/event/1/query'
r = requests.get(url, params=payload)

df = pd.read_csv(StringIO(r.text))
print(df.shape)
df.head()

### Generate Animated Map with `folium`

https://nbviewer.jupyter.org/github/python-visualization/folium/blob/master/examples/Plugins.ipynb#Timestamped-GeoJSON

In [ ]:
# get faults
r = requests.get('https://raw.githubusercontent.com/'
                 'fraxen/tectonicplates/master/GeoJSON/'
                 'PB2002_boundaries.json')

fault_features = r.json()['features']

In [ ]:
features = [
    {
        'type': 'Feature',
        'geometry': {
            'type': 'Point',
            'coordinates': [r['longitude'], r['latitude']],
        },
        'properties': {
            'time': r['time'][0:-1],
            'popup': (
                f"<strong>Time:</strong> {r['time']}<br>"
                f"<strong>Place:</strong> {r['place']}<br>"
                f"<strong>Magnitude:</strong> {r['mag']} {r['magType']}<br>"
                f"<strong>Depth:</strong> {r['depth']}<br>"
            ),
            'icon': 'circle',
            'iconstyle': {
                'fillOpacity': 0.5,
                'stroke': 0,
                'radius': r['mag'] * 2.5
            },
        }
    } for i, r in df.iterrows()
]

m = folium.Map(
#     location=()
    tiles='CartoDBpositron',
#     zoom_start=1,
#     no_wrap=True,
    min_zoom=1.5,
    max_zoom=5,
    world_copy_jump=True,
)

# add faults
folium.GeoJson(
    {
        'type': 'FeatureCollection',
        'features': fault_features,
    },
    style_function = lambda x: {
        'color': 'red',
        'weight': 0.5,
    }
).add_to(m)

plugins.TimestampedGeoJson(
    {
        'type': 'FeatureCollection',
        'features': features
    },
    period='PT6H', # six hour
    time_slider_drag_update=True,
    duration='PT12H',
    date_options='YYYY-MM-DD HH UTC'
).add_to(m)

folium.plugins.Fullscreen(
    position='topright',
    force_separate_button=True,
).add_to(m)

# m.save('earthquakes.html')
m

## Missing Data

#### Decreasing Reported Events in Previous 30 Days

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('seaborn')

In [ ]:
payload = {
    'format': 'csv', 
    'starttime': '2019-01-01',  # default last 30 days
    'minmagnitude': 4,  # default null
    'limit': None,  # default null, returns 404 over 20,000
}
url = 'https://earthquake.usgs.gov/fdsnws/event/1/query'
r = requests.get(url, params=payload)

df = pd.read_csv(StringIO(r.text))
df['time'] = pd.to_datetime(df['time'])
df['updated'] = pd.to_datetime(df['updated'])

print(df.shape)

In [ ]:
df['week_of'] = (df['time'].dt.floor('D') - pd.to_timedelta(df['time'].dt.dayofweek, unit='d'))
df['mag_rnd'] = df['mag'].apply(np.floor) # floor round to nearest 0.5 mag

week_mag_counts = (df.loc[(df['week_of'].min() < df['week_of']) & 
                          (df['week_of'] < df['week_of'].max()) &
                          (df['mag_rnd'] <= 6)]
                     .groupby(['mag_rnd', 'week_of'])
                     .size())

In [ ]:
fig, ax = plt.subplots(figsize=(8,4))
week_mag_counts.unstack(level=0).plot(ax=ax)

ax.legend(title='Magnitude')
ax.set_xlabel('Date (events grouped by week)')
ax.set_ylabel('Number of Events')

ax.set_ybound(lower=0)
ax.set_facecolor('#D4DADC')

for item in [ax.xaxis.label, ax.yaxis.label]:   
    item.set_fontweight('bold')

fig.savefig(
    '../ds-application/app/static/recency_freq.png', 
    facecolor=fig.get_facecolor(),
    dpi=160,
    transparent=False, 
    bbox_inches='tight',
);

In [ ]:
df['days_to_update'] = (df['updated'] - df['time']).dt.days

df_to_hist = pd.DataFrame({k: v.reset_index(drop=True) for k, v in df.loc[df['time'] > '2018'].groupby('mag_rnd')['days_to_update']})

fig, ax = plt.subplots(figsize=(8,4))
df_to_hist.plot.hist(stacked=True, ax=ax, bins=16)

ax.legend(title='Magnitude')
ax.set_xlabel('Age of Record when Updated (Days)')
ax.set_ylabel('Number of Records')

ax.set_ybound(lower=0)
# fig.set_facecolor('#FAFAF8')
ax.set_facecolor('#D4DADC')

for item in [ax.xaxis.label, ax.yaxis.label]:
#     item.set_color('#8B99A4')
    item.set_fontweight('bold')

fig.savefig(
    '../ds-application/app/static/update_dist.png', 
    facecolor=fig.get_facecolor(),
    dpi=160,
    transparent=False,
    bbox_inches='tight',
);

### Download USGS Data 1989-2019

In [ ]:
from dateutil.parser import parse
import requests
import pandas as pd
from io import StringIO

pd.options.display.max_columns = None

def dl_quake_data(start_date, end_date, page_limt=10000):
    start_date = parse(start_date).isoformat()
    end_date = parse(end_date).isoformat()
    payload = {
        'format': 'csv',
        'starttime': start_date,
        'endtime': end_date,
        'minmagnitude': 2,
        'limit': page_limt,
        'orderby': 'time-asc',
    }
    url = 'https://earthquake.usgs.gov/fdsnws/event/1/query'
    r = requests.get(url, params=payload)
    
    if r.status_code != 200:
        print('Error', r.status_code, r.url)
        return False
    
    df = pd.read_csv(StringIO(r.text))
    
    dt_min = df['time'].iloc[0]
    dt_max = df['time'].iloc[-1]
    
    fn = (f'{parse(dt_min).strftime("%Y-%m-%d")}_'
          f'{parse(dt_max).strftime("%Y-%m-%d")}')
    df.to_csv(f'data/{fn}.csv', index=False)
    
    print(fn)
    
    if len(df) == page_limt:
         dl_quake_data(start_date=dt_max,
                       end_date=end_date)
    
    return True

In [ ]:
# done '1989-01-01' to '2019-01-01'
# dl_quake_data('1990-11-15', '1999-01-01', 5000)

In [ ]:
from pathlib import Path

dfs = []
for csv in Path('data').iterdir():
    dfs.append(pd.read_csv(csv))
    
df = pd.concat(dfs)
df = df.drop_duplicates(['id'])

df['time'] = pd.to_datetime(df['time'])
df['updated'] = pd.to_datetime(df['updated'])

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


def OLS_mc_a_b(df, remove_tail=True):
    df = df.copy()
    df['mag'] = df['mag'].round(1)
    
    if remove_tail:
        df = df.loc[df['mag']<6]
    
    n_steps = int((df['mag'].max() - df['mag'].min()) * 10) + 1
    mag_range = np.linspace(df['mag'].min(), df['mag'].max(), n_steps)
    mag_range = [round(x, 2) for x in mag_range]
    
    counts = pd.Series(  # cumulative sum
        index=mag_range,
        data=[sum(df['mag'] >= x) for x in mag_range]
    )

    best_score = 0
    best_m_c, a, b = [None] * 3

    for M_c in np.arange(df['mag'].min(), 4.6, 0.1):
        M_c = round(M_c, 2)
        data = counts.loc[counts.index>=M_c]
        X = np.array(data.index).reshape(-1, 1)
        y = np.log10(data.values)
        reg = LinearRegression().fit(X, y)
        score = reg.score(X, y)

        if score > best_score:
            best_m_c = M_c
            best_score = score
            a = reg.intercept_
            b = -1 * reg.coef_[0]

    return best_m_c, best_score, a, b

In [ ]:
df = df.copy()
df['mag'] = df['mag'].round(1)

n_steps = int((df['mag'].max() - df['mag'].min()) * 10) + 1
mag_range = np.linspace(df['mag'].min(), df['mag'].max(), n_steps)
mag_range = [round(x, 2) for x in mag_range]

counts = pd.Series(  # cumulative sum
    index=mag_range,
    data=[sum(df['mag'] >= x) for x in mag_range]
)

In [ ]:
best_m_c, best_score, a, b = OLS_mc_a_b(df, remove_tail=False)

In [ ]:
fig, ax = plt.subplots(figsize=(8,4))

np.log10(counts).plot(ax=ax, label='Observed Events')
ax.plot([2, 9], [(a - b*x) for x in [2, 9]], label='Expected Events')
ax.vlines(best_m_c, 0, 9, linestyle='--', label='Magnitude Threshold')

ax.legend()
ax.set_xlabel('Magnitude')
ax.set_ylabel('Number of Events  ≥  Magnitude')

ax.set_ybound(lower=0, upper=8.5)
ax.set_facecolor('#D4DADC')
ax.set_yticklabels([0] + [f"$10^{x}$" for x in range(1, 10)])

for item in [ax.xaxis.label, ax.yaxis.label]:
#     item.set_color('#8B99A4')
    item.set_fontweight('bold')

fig.savefig(
    '../ds-application/app/static/gr_law.png', 
    facecolor=fig.get_facecolor(),
    dpi=160,
    transparent=False,
    bbox_inches='tight',
);

### Group Earthquake Events to Region

This is done because different regions have different detective power for small magnitude eathquakes. Here I generate a grid of equi-distant points around the globe before clustering earthquake events to the nearest node.

As improvements, I could define these regions algorithmically (like with [HDBSCAN](https://hdbscan.readthedocs.io/en/latest/index.html), [clusterpy](https://github.com/clusterpy/), [Moran'sI](https://en.wikipedia.org/wiki/Moran's_I)) based on earthquake characteristics in each regions. It may be that a special model already exists for this purpose [in extant research](https://www.researchgate.net/publication/260702383_A_detailed_seismic_zonation_model_for_shallow_earthquakes_in_the_broader_Aegean_area). Alternatively, knowing where seismic stations are located and how sensitive they are could help inform region definitions.

In [ ]:
# import hdbscan

# downsamp = df[['latitude', 'longitude']].sample(frac=0.05)
# clusterer = hdbscan.HDBSCAN(metric='haversine')
# clusterer.fit(downsamp)

In [ ]:
def gen_equidist_pts(num_pts=4000):
    indices = np.arange(0, num_pts, dtype=float) + 0.5

    phi = np.arccos(1 - 2*indices/num_pts)
    theta = np.pi * (1 + 5**0.5) * indices

    x = np.cos(theta) * np.sin(phi)
    y = np.sin(theta) * np.sin(phi)
    z = np.cos(phi)

    df = pd.DataFrame({'id':range(num_pts),
                       'latitude':  180 * np.arcsin(z / 1) / np.pi, 
                       'longitude': 180 * np.arctan2(y, x) / np.pi})
    
    return df

In [ ]:
m = folium.Map(
    tiles='CartoDBpositron',
    world_copy_jump=False,
    zoom_start=1,
    no_wrap=True,
)

for i, r in gen_equidist_pts().iterrows():
    folium.CircleMarker(
        (r['latitude'], r['longitude']),
        stroke=False,
        radius=2,
        fill=True,
        fill_opacity=0.75
    ).add_to(m)

plugins.Fullscreen(
    position='topright', 
    force_separate_button=True
).add_to(m)

m.save('../ds-application/app/static/eq_pts.html')

### Assign Quake Events to Cluster Centers

In [ ]:
from sklearn.neighbors import DistanceMetric

def nearest_left_merge(left_df, right_df, latlon_cols=['latitude', 'longitude'], step=50000):
    """For all rows of the left dataframe, merge in the nearest row from the right
    dataframe.
    
    left: 2D array of [lat, long]
    right: 2D array of [lat, long]
    """
    left_df = left_df.copy()
    right_df = right_df.copy()
    right_df = right_df.reset_index(drop=True)
    
    dist = DistanceMetric.get_metric('haversine') # requires radians
    left_rads = left_df[latlon_cols].values * np.pi / 180
    rght_rads = right_df[latlon_cols].values * np.pi / 180

    nearest_centers = []

    i = 0
    while i < len(left_df):
        dists = dist.pairwise(left_rads[i:i+step], rght_rads)
        indices = np.argmin(dists, axis=1)  # indices of minimum distance node
        dists = dists[np.arange(len(dists)), indices].reshape(-1, 1)
        result = np.hstack([right_df.values[indices], dists])
        nearest_centers.append(result)
        i += step
    
    df = pd.DataFrame(np.vstack(nearest_centers), 
                      columns=list(right_df) + ['dist'])
    df['dist'] = df['dist'] * 6371  # to km
    
    return df

In [ ]:
clust_df = gen_equidist_pts(num_pts=4000)

In [ ]:
# To filter down the number of regions, first determine which
# regions have ever had a 5.5+ quake
nearest_nodes = nearest_left_merge(df.loc[df['mag'] >= 5.5], clust_df)

In [ ]:
max_dist = nearest_nodes['dist'].max()
print(max_dist)
nearest_nodes['dist'].hist(bins=50);

In [ ]:
nearest_nodes.groupby(['id']).size().hist(bins=30);

In [ ]:
# filter to regions that have ever had a 5.5+ quake
clust_df = clust_df.loc[clust_df['id'].isin(nearest_nodes['id'])]
print(clust_df.shape)

In [ ]:
881/4000

In [ ]:
m = folium.Map(
    tiles='CartoDBpositron',
    world_copy_jump=False,
    zoom_start=1,
    no_wrap=True,
)

for i, r in clust_df.iterrows():
    folium.CircleMarker(
        (r['latitude'], r['longitude']),
        stroke=False,
        radius=2,
        fill=True,
        fill_opacity=0.75
    ).add_to(m)

plugins.Fullscreen(
    position='topright', 
    force_separate_button=True
).add_to(m)

m.save('../ds-application/app/static/eq_pts_filtered.html')

In [ ]:
# compute all events
all_nearest_nodes = nearest_left_merge(df, clust_df)

In [ ]:
# join
df = df.reset_index(drop=True)
df = df.join(all_nearest_nodes, rsuffix='_clust')

In [ ]:
df.groupby('id_clust').size().hist(bins=100);

In [ ]:
df = df.loc[df['dist'] <= max_dist]

In [ ]:
df['dist'].hist(bins=100);

In [ ]:
# weeks spanned by dataset
n_weeks = len(pd.date_range(df['time'].min().round('D'),
                            df['time'].max().round('D'), 
                            freq='W-SUN'))

In [ ]:
# n weeks in which a mag 5.5+ occured, by node
n_weeks_mag55 = (df.loc[df['mag'] >= 5.5]
                   .groupby(['id_clust', 
                             pd.Grouper(key='time', freq='W-SUN')])
                   .size()
                   .reset_index()
                   .groupby('id_clust')
                   .size())

In [ ]:
mag55_liklihoods = ((n_weeks_mag55 / n_weeks) * 100).round(1)

In [ ]:
mag55_liklihoods.hist();

In [ ]:
mag55_liklihoods.name = '%_liklihood'
clust_df = clust_df.merge(mag55_liklihoods.reset_index(), left_on='id', right_on='id_clust')

In [ ]:
# add zeros back in

In [ ]:
og_clust_df = gen_equidist_pts(num_pts=4000)
og_clust_df = og_clust_df.loc[~og_clust_df['id'].isin(clust_df['id'])]
og_clust_df['%_liklihood'] = 0

In [ ]:
clust_df = clust_df.append(og_clust_df, sort=True)

In [ ]:
# assess m_c for each node
results = []
for n_id, n_df in df.groupby('id_clust'):
    best_m_c, best_score, a, b = OLS_mc_a_b(n_df, remove_tail=False)
    results.append({
        'id_clust': n_id,
        'best_m_c': best_m_c,
        'best_score': best_score,
        'a': a,
        'b': b,
    })
    
results = pd.DataFrame(results)

In [ ]:
results['best_m_c'].hist();

In [ ]:
results['best_score'].hist(bins=100);

In [ ]:
def linear_gradient(start_hex, finish_hex="#FFFFFF", n=10, return_hex=True):
    '''Generate a gradient list of (n) colors

    start_hex, finish_hex: full six-digit color string ("#FFFFFF")
    '''

    s = hex_to_RGB(start_hex)
    f = hex_to_RGB(finish_hex)

    RGB_list = [s]
    # Calcuate a color at each evenly spaced value of t from 1 to n
    for t in range(1, n):
        # Interpolate RGB vector for color at the current value of t
        curr_vector = []
        for j in range(3):
            rgb_part = int(s[j] + (float(t) / (n-1)) * (f[j] - s[j]))
            curr_vector.append(rgb_part)

        # Add it to our list of output colors
        RGB_list.append(curr_vector)

    if return_hex:
        RGB_list = [RGB_to_hex(color) for color in RGB_list]

    return RGB_list

def RGB_to_hex(RGB):
    """[255,255,255] -> '#FFFFFF'"""
    # Components need to be integers for hex to make sense
    RGB = [int(x) for x in RGB]
    return "#"+"".join(["0{0:x}".format(v) if v < 16 else
                        "{0:x}".format(v) for v in RGB])

def hex_to_RGB(hex):
    """'#FFFFFF' -> [255,255,255]"""
    # Pass 16 to the integer function for change of base
    return [int(hex[i:i+2], 16) for i in range(1,6,2)]

In [ ]:
n_levels = 10
colors = linear_gradient('#f3bd90', '#d7191c', n_levels)
results['color'] = pd.cut(results['best_score'],
                          bins=n_levels, labels=colors)

m = folium.Map(
    tiles='CartoDBpositron',
    world_copy_jump=False,
    zoom_start=1,
    no_wrap=True,
)

for i, r in clust_df.merge(results, on='id_clust').iterrows():
    folium.CircleMarker(
        (r['latitude'], r['longitude']),
        stroke=False,
        radius=2,
        fill=True,
        fill_color=r['color_y'],
        fill_opacity=0.75
    ).add_to(m)

plugins.Fullscreen(
    position='topright', 
    force_separate_button=True
).add_to(m)

# m.save('../ds-application/app/static/eq_pts_filtered.html')
m

#### Colored Circle Map

In [ ]:
n_levels = 10
colors = linear_gradient('#f3bd90', '#d7191c', n_levels)
clust_df['color'] = pd.cut(clust_df['%_liklihood'],
                           bins=n_levels, labels=colors)

m = folium.Map(
    tiles='CartoDBpositron',
    world_copy_jump=True,
)

# add faults
for i, r in clust_df.loc[clust_df['%_liklihood'] > 0].iterrows():
#     if r['%_liklihood'] == 0:
#         continue
    folium.CircleMarker(
        (r['latitude'], r['longitude']),
        radius=4,
        stroke=False,
        fill=True,
        fill_color=r['color'],
        fill_opacity=0.75,
        tooltip=str(r['%_liklihood']),
    ).add_to(m)

m.save('test.html')

#### Contour Map

In [ ]:
# adapted from https://www.tjansson.dk/2018/10/contour-map-in-folium/
import json
import branca
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
import geojsoncontour
import scipy as sp
import scipy.ndimage
 
# Setup colormap
colors = linear_gradient('#f3bd90', '#d7191c', 15)
vmin   = 0
vmax   = 15
levels = len(colors)
cm     = branca.colormap.LinearColormap(colors, vmin=vmin, vmax=vmax).to_step(levels)
 
# The original data
x_orig = np.asarray(clust_df['longitude'].tolist())
y_orig = np.asarray(clust_df['latitude'].tolist())
z_orig = np.asarray(clust_df['%_liklihood'].tolist())
 
# Make a grid
x_arr          = np.linspace(-180, 180, 1000)
y_arr          = np.linspace( -90,  90, 1000)
x_mesh, y_mesh = np.meshgrid(x_arr, y_arr)
 
# Grid the values
z_mesh = griddata((x_orig, y_orig), z_orig, (x_mesh, y_mesh), method='linear')
 
# Gaussian filter the grid to make it smoother
sigma = [3, 3]
z_mesh = sp.ndimage.filters.gaussian_filter(z_mesh, sigma, mode='constant')

# Create the contour
contourf = plt.contourf(x_mesh, y_mesh, z_mesh, levels, alpha=0.5, colors=colors, 
                        linestyles='None', vmin=vmin, vmax=vmax);
 
# Convert matplotlib contourf to geojson
geojson = geojsoncontour.contourf_to_geojson(
    contourf=contourf,
    min_angle_deg=3.0,
    ndigits=5,
    stroke_width=1,
    fill_opacity=0.5,
)

geojson = json.loads(geojson)
geojson['features'] = [feat for feat in geojson['features'] 
                       if feat['properties']['title'] != '0.00 ']
 
# Set up the folium plot
m = folium.Map( 
    zoom_start=1, 
    tiles="cartodbpositron",
)

# Plot the contour plot on folium
folium.GeoJson(
    geojson,
    style_function=lambda x: {
        'color':     x['properties']['stroke'],
        'weight':    0.5,
        'opacity':   1,
        'fillColor': x['properties']['stroke'],
        'fillOpacity': float(x['properties']['title'])*15/100 + 0.1,
    },
    tooltip = folium.GeoJsonTooltip(
        fields=['title'],
        aliases=['% Likelihood']
    ),
).add_to(m)

# add faults
folium.GeoJson(
    {
        'type': 'FeatureCollection',
        'features': fault_features,
    },
    style_function = lambda x: {
        'color': 'grey',
        'weight': 0.5,
    }
).add_to(m)

# Add the colormap
cm.caption = 'Percent Likelihood of Mag 5.5+ in any 7 Day Period'
m.add_child(
    cm,
)

# Fullscreen mode
plugins.Fullscreen(
    position='topright', 
    force_separate_button=True
).add_to(m)

# Plot the data
m.save('../ds-application/app/static/likelihoods.html')
m

### Filter to clusters with >= 2000 observations

In [ ]:
clust_cnts = df.groupby(['clust_id']).size()
clust_cnts = clust_cnts.loc[clust_cnts >= 2000]

clust_df = clust_df.loc[clust_df['clust_id'].isin(clust_cnts.index)]
df = df.loc[df['clust_id'].isin(clust_cnts.index)]

In [ ]:
clust_cnts.shape

### Map Cluster Sizes

In [ ]:
import folium
import folium.plugins

m = folium.Map(tiles='CartoDBpositron')
clust_cnts = df.groupby(['clust_id', 'clust_lat', 'clust_lon']).size()

for i, v in clust_cnts.items():
    folium.CircleMarker(
        i[1:], 
        radius=np.sqrt((v/100) / np.pi) * 2,
        fill=True,
        fill_opacity=0.5,
        stroke=0,
        popup=f"id:{i[0]}<br>events: {v}",
    ).add_to(m)

folium.plugins.Fullscreen(
    position='topright',
    force_separate_button=True,
).add_to(m)

m

### Identify Cut-Off Magnitude for Each Node, `M_c`

Here I compare OLS and Mode methods. For a more robust method, see [Bayesian method](https://medium.com/the-history-risk-forecast-of-perils/exploring-the-fascinating-world-of-incomplete-seismicity-data-part-i-ii-bayesian-inference-386338b43b71).

In [ ]:
def fit_GR_OLS(node_df):
    n_steps = int((node_df['mag'].max() - node_df['mag'].min()) * 10) + 1
    mag_range = np.linspace(
        node_df['mag'].min(), 
        node_df['mag'].max(), 
        n_steps)
    mag_range = [round(x, 2) for x in mag_range]

    counts = pd.Series(
        index=mag_range,
        data=[sum(node_df['mag'] >= x) for x in mag_range]
    )

    X = np.array(counts.index).reshape(-1, 1)
    y = np.log10(counts.values)
    reg = LinearRegression().fit(X, y)
    a = reg.intercept_
    b = -1 * reg.coef_[0]
    mse = mean_squared_error(y, reg.predict(X))
    
    return a, b, mse


def mode_m_c_a_b(node_df):
    node_df = node_df.copy()
    node_df['mag'] = node_df['mag'].round(1)
    node_df = node_df.loc[node_df['mag']<6]
    m_c = node_df['mag'].mode()[0]
    node_df = node_df.loc[node_df['mag']>=m_c]
    a, b, _ = fit_GR_OLS(node_df)
    
    return m_c, a, b

In [ ]:
clust_id = 609
node_df = df.loc[df['clust_id']==clust_id]

In [ ]:
# OLS method
m_c, a, b = OLS_mc_a_b(node_df)

fig, ax = plt.subplots()
node_df['mag'].hist(bins=50, ax=ax)
plt.axvline(x=m_c, linewidth=2, color='r');

In [ ]:
# mode method
m_c, a, b = mode_m_c_a_b(node_df)

fig, ax = plt.subplots()
node_df['mag'].hist(bins=50, ax=ax)
plt.axvline(x=m_c, linewidth=2, color='r');

In [ ]:
# apply mode method
clust_df['m_c'], clust_df['a'], clust_df['b'] = np.nan, np.nan, np.nan

for clust_id in df['clust_id'].unique():
    node_df = df.loc[df['clust_id']==clust_id]
    m_c, a, b = mode_m_c_a_b(node_df)
    clust_df.loc[clust_df['clust_id']==clust_id, ['m_c', 'a', 'b']] = m_c, a, b

In [ ]:
clust_df['m_c'].hist();

### Filter Earthquake events by `m_c`

In [ ]:
df = df.merge(clust_df[['clust_id', 'm_c', 'a', 'b']], on='clust_id')

In [ ]:
df = df.query("mag >= m_c")

### Timeseries Feature Engineering

#### How Many Samples?

In literature, 50 of the most recent quakes were analyzed when the magnitude cutoff was 4.0. However, some regions have a lower threshold, allowing for exponentially more events. Therefore, I will take exponentially more events when calculated features for those regions:

In [ ]:
def get_n_samples(m_c):
    # From research:
    #  m_c=4, y=50
    # Extrapolated to the mc=0 case, with exponential increase
    #  m_c=0, y=500000
    # log10(n_events) = slope * m_c + y_int
    y_int = np.log10(500000)
    slope = -1 # by definition of logarithm

    return int(round(10 ** (slope * m_c + y_int)))

In [ ]:
df.shape

In [ ]:
# for each node, at weekly interval, collect past n events, predict whether 5.5+ will occur in the next 30 days
min_date = df['time'].min().round('D')
max_date = df['time'].max().round('D')
strt_sunday = min_date - pd.to_timedelta(min_date.dayofweek, unit='d')
end_sunday = (max_date - pd.to_timedelta(max_date.dayofweek, unit='d') - 
              pd.to_timedelta(30, unit='d')) # leave 30 day buffer at end to assess whether quake occured
sampling_periods = pd.date_range(strt_sunday, end_sunday, freq='W')

In [ ]:
x_df = pd.DataFrame(columns=['clust_id','sample_date',
                             'T_days','M_mean','dEsq',
                             'a_lsq','b_lsq','a_mlk', 'b_mlk','mse',
                             'diff_M_max_obs_exp','mu','c',
                             'quake_next30'])

for node_id, node_alltime_df in df.loc[df['clust_id'].isin([609, 554])].groupby(['clust_id']):
    print(node_id)
    
    N_events = get_n_samples(node_alltime_df['mag'].min())
    
    for sample_date in sampling_periods:
        # check if 5.5+ mag quake happend in next 30 days
        condition = ((sample_date < node_alltime_df['time']) & 
                     (node_alltime_df['time'] < (sample_date + pd.to_timedelta(30, unit='d'))))
        quake_next30 = sum(node_alltime_df.loc[condition, 'mag'] >= 5.5) > 0
        
        # take last 100 events, compute features
        node_df = node_alltime_df.loc[node_alltime_df['time']<=sample_date]
        
        if len(node_df) < N_events:
            continue
        
        node_df = node_df[-N_events:]

        # T: time period in days
        T = (node_df['time'].max() - node_df['time'].min()).days

        # M_mean: mean Magnitude
        M_mean = node_df['mag'].mean()

        # dEsq: seismic energy release
        dEsq = np.sum(
            np.sqrt(
                np.power(
                    np.array([10]*len(node_df)), 
                    11.8+1.5*node_df['mag']
                )
            )
        ) / T

        # a and b values (2 methods) and MSE from GR law
        n = len(node_df)
        # some issues in here:
        a_lsq, b_lsq, mse = fit_GR_OLS(node_df)
        
        b_mlk = np.log10(np.e) / (node_df['mag'].mean() - node_df['mag'].min())
        a_mlk = np.log10(n) + b_mlk * node_df['mag'].min()

        # difference between the maximum observed and the maximum expected
        diff_M_max_obs_exp = node_df['mag'].max() - a_lsq / b_lsq
        
        # mean and stdev time between mag 4.5 & 5 events
        diffs = node_df.loc[(4.5 <= node_df['mag']) & (node_df['mag'] <= 5), 'time'].diff()

        mu = diffs.mean().total_seconds()
        c = diffs.std().total_seconds() / mu
  
        # Maximum magnitude in last seven days
        # date_7days = node_df['time'].max() - pd.to_timedelta(7, unit='d')
        # x_6i = node_df.loc[node_df['time'] > date_7days, 'mag'].max()

        x_df = x_df.append(pd.DataFrame({
            'clust_id': [node_id],
            'sample_date': [sample_date],
            'T_days': [T],
            'M_mean': [M_mean],
            'dEsq': [dEsq],
            'a_lsq': [a_lsq],
            'b_lsq': [b_lsq],
            'a_mlk': [a_mlk],
            'b_mlk': [b_mlk],
            'mse':[mse],
            'diff_M_max_obs_exp': [diff_M_max_obs_exp],
            'mu': [mu],
            'c': [c],
            'quake_next30': [quake_next30],
        }))

In [ ]:
x_df.groupby(['clust_id']).size()

In [ ]:
x_df.groupby(['clust_id'])['quake_next30'].value_counts(normalize=True)

In [ ]:
x_df.loc[(x_df['clust_id']==554) & 
         (x_df['quake_next30']==True)]

In [ ]:
# bad temporal patterns 598, 591, 557, 543, 536, 502, 448

In [ ]:
df.loc[df['clust_id']==598, 'time'].dt.year.hist(bins=20);